In [ ]:
import re, mmap, textwrap
from datetime import datetime, timedelta

In [ ]:
pattern = re.compile('((\d{4})\/(\d{2})\/(\d{2})\/ (\d{2}):(\d{2}):(\d{2})\nSTARTING RUN (\d{3}) - RUNTYPE=(CALIBRATION|PULSER|TEST|RUN)\nFacility: FRIB		Experiment: 21010\nBeam: (.*)		Beam energy: (.*)		Target: (.*)\nTriggers: (.*)\nShifters: (.*)\n?(Comments: (.*))?)?\n?((\d{4})\/(\d{2})\/(\d{2})\/ (\d{2}):(\d{2}):(\d{2})\nMID-RUN COMMENTS FOR RUN (\d{3})\nShifters: (.*)\n(Comments: (.*))){0,}\n?((\d{4})\/(\d{2})\/(\d{2})\/ (\d{2}):(\d{2}):(\d{2})\nSTOPPING RUN (\d{3})\nShifters: (.*)\n?(Comments: (.*))?)?\n((\d{4})\/(\d{2})\/(\d{2})\/ (\d{2}):(\d{2}):(\d{2})\nIN-BETWEEN RUN COMMENTS\nShifters: (.*)\n(Comments: (.*))){0,}')

with open("logs/run_log.txt", "r+") as f:
    data = mmap.mmap(f.fileno(), 0)
    result = pattern.findall(data.read().decode())

In [ ]:
sheet = open("run_log.csv", "w")
sheet.write('"Start time\n(EDT)";"Duration";"Run no.";"Run type";"Beam";"Beam energy";"Target";"Shifters";"Comments (given at start of run,\nif not otherwise specified)";"Triggers" \n')

def cest2edt(y, m, d, h, M, s):
    result = datetime(int(y), int(m), int(d), int(h), int(M), int(s)) - timedelta(hours=6)
    return result.year, result.month, result.day, result.hour, result.minute, result.second

#no_stop = [45, 69, 71, 72, 93, 111, 119]
#two_inbetweens = [32->33, 79->80, 82->83, 111->112]

i = 0
for match in result:
    if len(match[0]) == 0 and len(match[38]) == 0:
        continue
    has_start = has_mid = has_end = has_post = False
    runno = runtype = triggers = ' '
    beam = beam_energy = target = '-'
    duration = 'N/A'
    if len(match[0]) != 0:
        has_start = True
        starty        = match[1]
        startm        = match[2]
        startd        = match[3]
        starth        = match[4]
        startM        = match[5]
        starts        = match[6]
        runno         = match[7]
        runtype       = match[8]
        beam          = match[9]
        beam_energy   = match[10]
        target        = match[11]
        triggers      = match[12]
        startshifters = match[13]
        startcomments = match[15]
    if len(match[16]) != 0:
        has_mid = True
        midy          = match[17]
        midm          = match[18]
        midd          = match[19]
        midh          = match[20]
        midM          = match[21]
        mids          = match[22]
        midshifters   = match[24]
        midcomments   = match[26]
    if len(match[27]) != 0:
        has_end = True
        endy          = match[28]
        endm          = match[29]
        endd          = match[30]
        endh          = match[31]
        endM          = match[32]
        ends          = match[33]
        endshifters   = match[35]
        endcomments   = match[37]
    if len(match[38]) != 0:
        has_post = True
        posty         = match[39]
        postm         = match[40]
        postd         = match[41]
        posth         = match[42]
        postM         = match[43]
        posts         = match[44]
        postshifters  = match[45]
        postcomments  = match[47]
    
    if has_start and has_end:
        duration = datetime(int(endy), int(endm), int(endd), 
                            int(endh), int(endM), int(ends)) \
                 - datetime(int(starty), int(startm), int(startd), 
                            int(starth), int(startM), int(starts))
    
    if has_start:
        start_time = '%04d-%02d-%02d\n%02d:%02d:%02d' % cest2edt(starty, startm, startd, starth, startM, starts)
        shifters   = startshifters
    elif has_post:
        start_time = '%04d-%02d-%02d\n%02d:%02d:%02d' % cest2edt(posty, postm, postd, posth, postM, posts)
        shifters   = postshifters
    else:
        print("Warning: No start time or shifters in match no. %d" % i)
        start_time = 'N/A'
        shifters   = 'N/A'
    
    comments = ""
    if has_start and len(startcomments) > 0:
        comments += startcomments
    if has_mid:
        if len(comments) > 0:
            comments += '\n'
        midduration = datetime(int(midy), int(midm), int(midd), 
                            int(midh), int(midM), int(mids)) \
                    - datetime(int(starty), int(startm), int(startd), 
                            int(starth), int(startM), int(starts))
        comments += 'EXTRA COMMENT %.1f MINUTES INTO RUN BY %s: ' \
                    % (midduration.seconds/60, midshifters) + midcomments
    if has_end and len(endcomments) > 0:
        if len(comments) > 0:
            comments += '\n'
        comments += 'END-OF-RUN COMMENT BY %s: ' % endshifters + endcomments
    if has_post and not has_start:
        if len(comments) > 0:
            comments += '\n'
        comments += 'IN-BETWEEN-RUN COMMENT: ' + postcomments
    comments = comments.replace('"', "'")
    comments = comments.replace('[LINEBREAK]', '\n')
    
    if len(comments) > 0:
        lines = textwrap.wrap(comments, 36, replace_whitespace=False)
        comments = ""
        for line in lines:
            comments += line + '\n'
        comments = comments[:-1]
    if len(triggers) > 0:
        lines = textwrap.wrap(triggers, 36, replace_whitespace=False)
        triggers = ""
        for line in lines:
            triggers += line + '\n'
        triggers = triggers[:-1]
    #if len(shifters) > 0:
    #    lines = textwrap.wrap(shifters, 16, replace_whitespace=False)
    #    shifters = ""
    #    for line in lines:
    #        shifters += line + '\n'
    #    shifters = shifters[:-1]
    
    sheet.write('"%s";"%s";"%s";"%s";"%s";"%s";"%s";%s;"%s";"%s" \n' \
                % (start_time, duration, runno, runtype, beam, beam_energy, target, shifters, comments, triggers))
    if has_post and has_start:
        start_time = '%04d-%02d-%02d\n%02d:%02d:%02d' % cest2edt(posty, postm, postd, posth, postM, posts)
        shifters   = postshifters
        comments = 'IN-BETWEEN-RUN COMMENT: ' + postcomments
        comments = comments.replace('"', "'")
        comments = comments.replace('[LINEBREAK]', '\n')
        if len(comments) > 0:
            lines = textwrap.wrap(comments, 36, replace_whitespace=False)
            comments = ""
            for line in lines:
                comments += line + '\n'
            comments = comments[:-1]
        sheet.write('"%s";"%s";"%s";"%s";"%s";"%s";"%s";%s;"%s";"%s" \n' \
                    % (start_time, 'N/A', ' ', ' ', '-', '-', '-', shifters, comments, ' '))
    
    i += 1
sheet.close()